In [1]:
# 问题分析
# 数据读取与数据处理
# 模型搭建与训练
# 模型验证
# 结果输出

# 1. 导入库

In [2]:
#安装相关依赖库 如果是windows系统，cmd命令框中输入pip安装，或在Jupyter notebook中!pip安装，参考上述环境配置
#!pip install pandas numpy cv2 torch torchvision codecs PIL glob
#---------------------------------------------------
#导入库
import os
import glob
from PIL import Image
import csv, time
import numpy as np

# pytorch相关
import torch
import torchvision
import torch.optim as optim
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.utils.data as data

C:\Users\BiXY\anaconda3\envs\mmcv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#输出cuda说明使用gpu，输出cpu说明使用cpu，最好使用gpu训练
print(device)

cuda


# 2. 数据读取与数据处理

In [4]:
# 自定义读取数据集
class ImageSet(data.Dataset):
    def __init__(
            self,
            images,
            labels,
            transform):
        self.transform = transform
        self.images = images
        self.labels = labels

    def __getitem__(self, item):
        imagename = self.images[item]
        
        # 防止文件出错，这里生成一个随机的照片
        try:
            image = Image.open(imagename)
            image = image.convert('RGB')
        except:
            image = Image.fromarray(np.zeros((256, 256), dtype=np.int8))
            image = image.convert('RGB')

        image = self.transform(image)
        return image, torch.tensor(self.labels[item])

    def __len__(self):
        return len(self.images)

In [5]:
import pandas as pd
import codecs

# 训练集标注数据
lines = codecs.open('train_label.csv').readlines()
train_label = pd.DataFrame({
    'image': ['train_image/' + x.strip().split(',')[0] for x in lines],
    'label': [x.strip().split(',')[1:] for x in lines],
})

# 将标签进行二值化处理
train_label['new_label'] = train_label['label'].apply(lambda x: int('0' in x))

# 数据扩增方法
trfs = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 训练集dataset和dataloder
# 这里我们使用前1000张图片进行训练，后续可以自行修改
train_dataset = ImageSet(train_label['image'].values,
                         train_label['new_label'].values,
                         trfs)
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=0,
    pin_memory=True,
)

# 测试集dataset和dataloder
test_images = glob.glob('./test_images/*')
test_dataset = ImageSet(test_images, [0] * len(test_images), trfs)

test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
)

In [6]:
for data in train_loader:
    break
    
for data in test_loader:
    break
      
 #如果出现BrokenPipeError: [Errno 32] Broken pipe，调整DataLoader中num_workers = 0
 #num_workers参数是指在进行数据集加载时，启用的线程数目，windows下多线程可能出问题

# 3. 模型搭建与训练

In [7]:
# 加载resnet18预训练模型
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, 2)
model = model.to('cuda') #使用GPU

# 模型优化器
optimizer = optim.SGD(model.parameters(), lr=0.001)

# 模型损失函数
loss = nn.CrossEntropyLoss()

# 接下来我们使用pytorch完成模型的训练：
# Python
# 设置迭代轮数epochs，可调整，轮数越多，所花时间越久
epochs = 5
for epoch in range(epochs):
    start_t = time.time()
    epoch_l = 0
    epoch_t = 0
    
    # 批量训练
    for batch_idx, batch in enumerate(train_loader):
        optimizer.zero_grad()
        image, label = batch
        image, label = image.to('cuda'), label.to('cuda')
        output = model(image) # 正向传播

        l = loss(output, label) # 计算损失
        l.backward()
        optimizer.step()

        batch_l = l.item()
        epoch_l += batch_l
        batch_t = time.time() - start_t
        epoch_t += batch_t
        start_t = time.time()
        
        # 打印loss
        if batch_idx % 10 == 0:
            print(l.item(), batch_idx, len(train_loader))

    epoch_t = epoch_t / len(train_loader)
    epoch_l = epoch_l / len(train_loader)
    print('...epoch: {:3d}/{:3d}, loss: {:.4f}, average time: {:.2f}.'.format(
        epoch + 1, epochs, epoch_l, epoch_t))

0.8650211095809937 0 63
0.4376973807811737 10 63
0.1396140456199646 20 63
0.24723991751670837 30 63
0.25899559259414673 40 63
0.3117246925830841 50 63
0.32275086641311646 60 63
...epoch:   1/  5, loss: 0.3932, average time: 0.35.
0.20990689098834991 0 63
0.49788349866867065 10 63
0.5804922580718994 20 63
0.31500667333602905 30 63
0.3174930512905121 40 63
0.31883499026298523 50 63
0.5544729232788086 60 63
...epoch:   2/  5, loss: 0.3707, average time: 0.14.
0.6120080947875977 0 63
0.1888469159603119 10 63
0.5087997913360596 20 63
0.3768598437309265 30 63
0.4384687840938568 40 63
0.5726935267448425 50 63
0.25547969341278076 60 63
...epoch:   3/  5, loss: 0.3692, average time: 0.14.
0.20117323100566864 0 63
0.3768639862537384 10 63
0.31902289390563965 20 63
0.37678638100624084 30 63
0.5640024542808533 40 63
0.4373927414417267 50 63
0.18015898764133453 60 63
...epoch:   4/  5, loss: 0.3683, average time: 0.14.
0.3803289830684662 0 63
0.17054255306720734 10 63
0.2537653148174286 20 63
0.256

# 4. 模型验证

In [8]:
model.eval()
to_prob = nn.Softmax(dim=1)
with torch.no_grad():
    imagenames, probs = list(), list()
    for batch_idx, batch in enumerate(test_loader):
        image, _ = batch
        image = image.to('cuda')
        pred = model(image)
        prob = to_prob(pred)
        prob = list(prob.data.cpu().numpy())
        probs += prob

# 5. 结果输出

In [9]:
import csv
with open('submission.csv', 'w',newline = '', encoding='utf8') as fp:
    writer = csv.writer(fp)
    writer.writerow(['imagename', 'defect_prob'])
    for imagename, prob in zip(test_images, probs):
        imagename = os.path.basename(imagename)
        writer.writerow([imagename, str(prob[1])])